## Interactive visualization the classification process

You might need to restart the colab notebook kernel, after pip install and each visualization display.

In [ ]:
!pip install ipympl

In [ ]:
%matplotlib widget

In [ ]:
from google.colab import output
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

output.enable_custom_widget_manager()

## SVM

In [ ]:


# Global training data (start with empty arrays)
X = np.empty((0, 2))
y = np.empty((0,))

# Set up a meshgrid for plotting the SVM decision function.
x_range = np.linspace(-6, 6, 300)
y_range = np.linspace(-6, 6, 300)
xx, yy = np.meshgrid(x_range, y_range)

# Create a figure and axis.
fig, ax = plt.subplots(figsize=(6, 5))

# Initialize the SVM classifier.
clf = SVC(kernel='linear', C=1.0)

def update_plot():
    """Retrain the SVM (if possible) and update the plot."""
    ax.clear()

    # Only fit if we have points from both classes.
    if len(np.unique(y)) >= 2:
        clf.fit(X, y)
        # Evaluate the decision function over the mesh grid.
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        # Plot filled contours for the decision regions.
        ax.contourf(xx, yy, Z, levels=[Z.min(), 0, Z.max()],
                    alpha=0.2, colors=['blue', 'red'])
        # Plot the decision boundary and margins:
        #   - Decision boundary: w·x + b = 0 (solid line)
        #   - Margins: w·x + b = ±1 (dashed lines)
        ax.contour(xx, yy, Z, levels=[-1, 0, 1],
                   colors='k', linestyles=['--', '-', '--'])

    # Plot the current training data (if any).
    if X.shape[0] > 0:
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolors='k')

    ax.set_xlim(x_range.min(), x_range.max())
    ax.set_ylim(y_range.min(), y_range.max())
    ax.set_xlabel("Feature 1")
    ax.set_ylabel("Feature 2")
    ax.set_title("Click to add points:\nLeft click = Class 1 (red), Right click = Class -1 (blue)")
    plt.draw()

def on_click(event):
    """Event handler for mouse clicks."""
    # Only process clicks inside the axes.
    if event.inaxes != ax:
        return
    # Determine label based on mouse button:
    # Left button (1) -> label 1, Right button (3) -> label -1.
    if event.button == 1:
        label = 1
    elif event.button == 3:
        label = -1
    else:
        return  # ignore other buttons

    global X, y
    # Append the new point.
    new_point = np.array([[event.xdata, event.ydata]])
    X = np.vstack([X, new_point])
    y = np.append(y, label)

    update_plot()

# Connect the click event handler.
fig.canvas.mpl_connect('button_press_event', on_click)

# Draw the initial empty plot.
update_plot()
plt.show()


## Logistic Regression

In [ ]:


# Global arrays for storing training data (initially empty).
X = np.empty((0, 2))
y = np.empty((0,))

# Create a meshgrid over the feature space.
x_range = np.linspace(-6, 6, 300)
y_range = np.linspace(-6, 6, 300)
xx, yy = np.meshgrid(x_range, y_range)

# Create the figure and axis.
fig, ax = plt.subplots(figsize=(6, 5))

# Initialize the logistic regression classifier.
clf = LogisticRegression()

def update_plot():
    """Retrain logistic regression (if possible) and update the plot."""
    ax.clear()

    # Only fit if there are at least two classes.
    if len(np.unique(y)) >= 2:
        clf.fit(X, y)
        # Evaluate the decision function over the grid.
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        # Plot filled contours for the decision regions.
        ax.contourf(xx, yy, Z, levels=[Z.min(), 0, Z.max()],
                    alpha=0.2, colors=['blue', 'red'])
        # Plot the decision boundary (where the decision function is 0).
        ax.contour(xx, yy, Z, levels=[0], colors='k', linestyles='-')

    # Plot the training data.
    if X.shape[0] > 0:
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolors='k')

    ax.set_xlim(x_range.min(), x_range.max())
    ax.set_ylim(y_range.min(), y_range.max())
    ax.set_xlabel("Feature 1")
    ax.set_ylabel("Feature 2")
    ax.set_title("Click to add points:\nLeft click = Class 1 (red), Right click = Class -1 (blue)")
    plt.draw()

def on_click(event):
    """Add a new data point based on the click and update the plot."""
    if event.inaxes != ax:
        return
    # Determine label: left click for class 1, right click for class -1.
    if event.button == 1:
        label = 1
    elif event.button == 3:
        label = -1
    else:
        return  # Ignore other buttons.

    global X, y
    new_point = np.array([[event.xdata, event.ydata]])
    X = np.vstack([X, new_point])
    y = np.append(y, label)
    update_plot()

# Connect the click event to the on_click handler.
fig.canvas.mpl_connect('button_press_event', on_click)

# Initial plot.
update_plot()
plt.show()


## Random Forest

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

# Global arrays for storing training data (start empty)
X = np.empty((0, 2))
y = np.empty((0,))

# Create a meshgrid for plotting over a specified range.
x_range = np.linspace(-6, 6, 300)
y_range = np.linspace(-6, 6, 300)
xx, yy = np.meshgrid(x_range, y_range)

# Create a figure and axis.
fig, ax = plt.subplots(figsize=(6, 5))

# Initialize the Random Forest classifier.
clf = RandomForestClassifier(n_estimators=100, random_state=42)

def update_plot():
    """Retrain the classifier and update the plot."""
    ax.clear()

    # Only fit if we have at least one sample from each class.
    if len(np.unique(y)) >= 2:
        clf.fit(X, y)
        # Compute the probability for class 1 over the mesh grid.
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        Z = Z.reshape(xx.shape)

        # Plot filled contours showing the probability distribution.
        # Colors go from blue (low probability for class 1) to red (high probability for class 1).
        ax.contourf(xx, yy, Z, levels=np.linspace(0, 1, 11),
                    alpha=0.2, cmap='bwr')
        # Plot the decision boundary at the 0.5 probability level.
        ax.contour(xx, yy, Z, levels=[0.5], colors='k', linestyles='-')

    # Plot the current training points.
    if X.shape[0] > 0:
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolors='k')

    ax.set_xlim(x_range.min(), x_range.max())
    ax.set_ylim(y_range.min(), y_range.max())
    ax.set_xlabel("Feature 1")
    ax.set_ylabel("Feature 2")
    ax.set_title("Random Forest Classifier\nClick to add points:\nLeft click = Class 1 (red), Right click = Class -1 (blue)")
    plt.draw()

def on_click(event):
    """Handle click events to add a new data point."""
    if event.inaxes != ax:
        return
    # Use left click for class 1 and right click for class -1.
    if event.button == 1:
        label = 1
    elif event.button == 3:
        label = -1
    else:
        return  # Ignore other mouse buttons.

    global X, y
    new_point = np.array([[event.xdata, event.ydata]])
    X = np.vstack([X, new_point])
    y = np.append(y, label)
    update_plot()

# Connect the mouse click event to the on_click function.
fig.canvas.mpl_connect('button_press_event', on_click)

# Draw the initial empty plot.
update_plot()
plt.show()
